##SETUP

In [2]:
import os
import time
import timeit
import pandas as pd
import random
import numpy as np
# Android ADB
from scrcpy import Client, const
from subprocess import check_output,Popen
import threading

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm
# internal
import bot_core
import importlib
importlib.reload(bot_core)

SLEEP_DELAY=0.1

# Start Scrcpy (once after PC restart)
start_scrcpy=True
if start_scrcpy:
    proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
    time.sleep(6) # <-- sleep for 6''
    proc.terminate() # <-- terminate the process


bot = bot_core.Bot()

OSError: [Errno subprocess] c:\Users\a430180\AppData\Local\Programs\Python\Python39\lib\site-packages\adbutils\binaries\adb.exe -s emulator-5554 push c:\Users\a430180\AppData\Local\Programs\Python\Python39\lib\site-packages\scrcpy\scrcpy-server-v1.24.jar /data/local/tmp/: "adb: error: failed to get feature set: device 'emulator-5554' not found\r\n"

In [19]:
bot.client.control.text("test")


b'\x01\x00\x00\x00\x04test'

## Game stats

In [159]:
# get stored mana 
#bot.getText(220,1360,90,50,new=False,digits=True)

# get unit mana 
bot.getText(450,1360,90,50,new=False,digits=True)

# Get upgrade mana
#bot.getText(20,1540,700,50,new=False,digits=True).split('6')


'70'

# Play Game

In [4]:
# Kill the game and restart
def restart_RR():
        bot.shell(f'input keyevent {const.KEYCODE_APP_SWITCH}') #Go app switch
        time.sleep(1)
        bot.click(781,80)# Clear all apps
        time.sleep(2)
        bot.shell(f'input tap 420 350') # re-open RR
        time.sleep(10)

def spam_unit():
    bot.getScreen()
    for i in range(200):
        bot.getScreen()
        print("Step,", i)
        # get stored mana 
        mana=bot.getText(220,1360,90,50,new=False,digits=True)
        if mana=='': mana=0
        # get unit mana 
        mana_cost=bot.getText(450,1360,90,50,new=False,digits=True)
        print(mana,mana_cost)
        if mana_cost=='' and  mana==0:
            time.sleep(1)
            continue
        if int(mana)>int(mana_cost):
            print(f"Bought for {mana_cost}!")
            bot.click(450,1360)
            time.sleep(0.1)
        else: time.sleep(1)

#battle_screen(start=True)
#spam_unit()

In [40]:
bot.getScreen()

In [5]:
restart_RR()

In [246]:
pos=bot.getXYByImage('fighting',new=False)
print(pos)
#if pos!=None:
 #   return


[669, 1359]


## Extra unit logic 

In [ ]:
##Extra unit logic 
# Try to merge any priestess
#if 'priest.png' in unit_list:
#    if df_groups['priest.png']<1:
#        bot.merge_unit(grid_df,'priest.png')

# Basic Loop

In [24]:
def try_merge():
    info=''
    names=bot.scan_grid(new=True)
    grid_df=bot_core.grid_status(names)
    df_split, df_groups, unit_list = bot_core.get_unit_count(grid_df)
    df_groups = df_groups.sort_values(ascending=False)
    # check if grid full
    if 'empty.png' in unit_list:
        # Merge if full board
        if df_groups['empty.png']<3:
            info='Merging!'
            unit_list.remove('empty.png')
            if 'hunter.png' in unit_list: unit_list.remove('hunter.png')
            for i in range(3):
                merge_tar = random.choice(unit_list)
                for j in range(3):
                    bot.merge_unit(grid_df,merge_tar)
                    time.sleep(0.2)
        else: info= 'need more units!'
    # If grid seems full, merge any
    else:
        info = 'Full Grid - Merging!'
        merge_tar = random.choice(unit_list)
        bot.merge_unit(grid_df,merge_tar)
    return grid_df,df_groups,info
    
def get_button_pos(df,button):
    button=button+'.png'
    pos=df[df['icon']==button]['pos [X,Y]'].reset_index(drop=True)[0]
    return pos

def battle_screen(start=False):
    # Scan screen for any key buttons
    button_df = bot.get_current_icons()
    avail_buttons=button_df[button_df['available']==True].reset_index(drop=True)
    if not avail_buttons.empty:
        # list of buttons
        button_names=avail_buttons['icon'].tolist()
        if 'fighting.png' in button_names and not '0cont_button.png' in button_names:
            return avail_buttons,'fighting'
        # Start pvp if homescreen
        if start and 'pvp_button.png' in button_names and 'battle_icon.png' in button_names:
            pvp_pos = get_button_pos(button_df,'pvp_button')
            bot.click_button(pvp_pos)
            return avail_buttons, 'home'
        # Check first button is clickable
        first_button=button_names[0].split('.')[0]
        if first_button in ['back_button','battle_icon','0cont_button','1quit']:
            button_pos=avail_buttons['pos [X,Y]'].tolist()[0]
            bot.click_button(button_pos)
            return avail_buttons,'menu'
    else:
        bot.shell(f'input keyevent {const.KEYCODE_BACK}') #Force back
        return button_df,'lost'
        # add recursive call to restart last recorse?
battle_screen()

(              icon  available    pos [X,Y]
 0  battle_icon.png       True  (359, 1419)
 1   pvp_button.png       True  (140, 1259),
 'menu')

In [25]:
wait=0
# Main loop
while(1):
    output = battle_screen(start=True)
    if output[1]=='fighting':
        wait = 0
        battle_bar=trange(10)
        for i in battle_bar:
                time.sleep(0.2)
                bot.click(450,1360)
                # Upgrade units
                bot.click(500,1500,delay_mult=4)
                #bot.click(200,1500)
                #bot.click(650,1500,delay_mult=4)
                grid_df,df_groups,info = try_merge()
                clear_output()
                display([f'{i+1}/10',output[1],info,grid_df,df_groups])
    else: 
        wait+=1
        if wait>40:
            output[1] = 'RESTARTING'
            restart_RR()
        print(output[1],'wait count:',wait)

['10/10',
 'fighting',
 'need more units!',
     grid_id           unit  probability position
 0    icon_0  cold_mage.png        557.0   [0, 0]
 1    icon_1      empty.png        999.0   [0, 1]
 2    icon_2      empty.png        999.0   [0, 2]
 3    icon_3  lightning.png        463.0   [0, 3]
 4    icon_4      empty.png        999.0   [0, 4]
 5    icon_5      empty.png        700.0   [1, 0]
 6    icon_6  lightning.png        444.0   [1, 1]
 7    icon_7   poisoner.png        477.0   [1, 2]
 8    icon_8      empty.png        700.0   [1, 3]
 9    icon_9      empty.png        700.0   [1, 4]
 10  icon_10  cold_mage.png        700.0   [2, 0]
 11  icon_11  cold_mage.png        700.0   [2, 1]
 12  icon_12      empty.png        700.0   [2, 2]
 13  icon_13      empty.png        700.0   [2, 3]
 14  icon_14  cold_mage.png        700.0   [2, 4],
 unit
 empty.png        8
 cold_mage.png    4
 lightning.png    2
 poisoner.png     1
 Name: unit, dtype: int64]

menu wait count: 1
menu wait count: 2
lost wait count: 3
home wait count: 4


KeyboardInterrupt: 

In [93]:
bot.click(460,1560)

In [243]:
#bot.getScreen()
# Battle icon
#bot.crop_img(360,1420,180,180,name='battle_icon.png')
# PVP button
#bot.crop_img(140,1260,200,80,name='pvp_button.png')
#back button
#bot.crop_img(60,1480,70,70,name='back_button.png')
#continue button
#bot.crop_img(380,1460,70,70,name='cont_button.png')
#fight_portal
bot.crop_img(670,1360,60,60,name='fighting.png')


In [78]:
bot.getXYByImage('battle_icon')